<a href="https://colab.research.google.com/github/MarshaGomez/DNN-Sketches-image-analysis/blob/main/Code/evaluate_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Presentation Results

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import time
import itertools
import os, shutil
import tensorflow as tf
import matplotlib.pyplot as plt

from random import random
from ipywidgets import Image
from numpy.linalg import norm
from IPython.display import display
from keras.models import Model
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint

# tensorflow version 2.4.0
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, InceptionV3

In [ ]:
zip_path = '/content/gdrive/Shareddrives/COMPUTER_VISION/MIM_zipped.zip'
!cp "{zip_path}" .
!unzip -q MIM_zipped.zip
!rm MIM_zipped.zip

## Setting up the project


### create Test (utile per quando faremo il training)

In [ ]:
#data loading
# reading from unzipped
BASE_DIR = "/content/content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV"
# FILELIST_PATH = BASE_DIR + "/filelist.txt"
SKETCHES_DIR = os.path.join(BASE_DIR, "sketches")
MIRFLICKR_DIR = os.path.join(BASE_DIR, "mirflickr/mirflickr25k")

#print(FILELIST_PATH)

#num_folders = os.listdir(SKETCHES_DIR)
#print(num_folders)

In [ ]:
print(os.listdir(os.path.join(SKETCHES_DIR, "png")))
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
      
copytree(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/all')

['rooster', 'head-phones', 'door handle', 'flashlight', 'panda', 'bottle opener', 'revolver', 'binoculars', 'piano', 'lightbulb', 'saxophone', 'snowboard', 'giraffe', 'filelist.txt', 'scissors', 'rabbit', 'pear', 'ship', 'helicopter', 'van', 'barn', 'tiger', 'dragon', 'ice-cream-cone', 'wheel', 'dolphin', 'cell phone', 'tree', 'squirrel', 'speed-boat', 'wheelbarrow', 'screwdriver', 'rifle', 'arm', 'bee', 'house', 'penguin', 'blimp', 'pretzel', 'motorbike', 'mosquito', 'hourglass', 'moon', 'spoon', 'pig', 'pizza', 'rollerblades', 'ant', 'cabinet', 'car (sedan)', 'book', 'bed', 'sailboat', 'candle', 'tooth', 'train', 'strawberry', 'palm tree', 'streetlight', 'present', 'angel', 'envelope', 'table', 'eye', 'bathtub', 'lion', 'fan', 'crocodile', 'castle', 'keyboard', 'crown', 'flying bird', 'human-skeleton', 'octopus', 'laptop', 'nose', 'parrot', 'umbrella', 'bookshelf', 'sponge bob', 'satellite', 'couch', 'bicycle', 'stapler', 'kangaroo', 'door', 'ear', 'mug', 'hot-dog', 'suv', 'fish', 'k

In [ ]:
test_images_per_class = 20 # le immagini sono al massimo 80
try:
  os.rename(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/train')
except:
  print('already renamed')

if not os.path.exists(f'{SKETCHES_DIR}/test'):
  os.mkdir(f'{SKETCHES_DIR}/test')
  for class_folder in os.listdir(f'{SKETCHES_DIR}/train'):
    if os.path.isdir(f'{SKETCHES_DIR}/train/{class_folder}'):
      images = os.listdir(f'{SKETCHES_DIR}/train/{class_folder}')
      os.mkdir(f'{SKETCHES_DIR}/test/{class_folder}')
      for i in range(test_images_per_class):
        os.rename(f'{SKETCHES_DIR}/train/{class_folder}/{images[i]}', f'{SKETCHES_DIR}/test/{class_folder}/{images[i]}')


### No Index definition and evaluation


In [ ]:
def average_precision(requested_label, result_labels, n_ground_truth = 80):
  """
  label ricercata, label ottenute, il numero di oggetti che ci sono quella label
  """
  correct_array = (requested_label == result_labels).astype(int)
  precision_array = [np.mean(correct_array[:k]) for k in range(1, correct_array.shape[0] + 1)]
  # print(precision_array * correct_array) # mi rimangono solo quelli a 1
  return np.sum(precision_array * correct_array) / n_ground_truth

In [ ]:
def mAP(index, features, n_queries = 250, n_labels = 250, img_per_labels = 80, mode = 'euclidean', check_first_label=True, n_ground_truth=None):
  sum = 0
  if not n_ground_truth:
    n_ground_truth = img_per_labels
  for i in range(n_queries):
    label = i % n_labels
    image_idx = ((i * img_per_labels) + int(random() * img_per_labels)) % (n_labels * img_per_labels)
    # print('QUERY')
    # print('index = ' + str(image_idx))
    # print('label =' + str(label))
    res = index.query(features[image_idx], n_ground_truth, mode = mode)
    # print('first label of resultset (must be equal to label) = ' + str(res['labels'][0]))
    if check_first_label:
      assert res['labels'][0] == label, 'deve essere della stessa label'
    a = average_precision(label, res['labels'], n_ground_truth)
    sum += a
  return sum / n_queries


In [ ]:
class NO_INDEX:
  def __init__(self):
    self.features = None
    self.ids = None
    self.labels = None

  def insert(self, features, ids, labels):
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    self.features = features
    self.ids = ids
    self.labels = labels
  
  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    top_k += 1
    if mode == 'euclidean':
      dist = norm(self.features - np.array(features), axis=1)
      idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
      k = {}
      # qua è più comodo avere array 1- dimensionali
      k['ids'] = self.ids[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['labels'] = self.labels[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['distances'] = dist[idx_partitioned][:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, dist.size)
      return k
    sim = np.sum(self.features * np.array(features), axis=1) / (norm(self.features, axis=1) * norm(np.array([features]), axis=1))
    idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
    k = {}
    # qua è più comodo avere array 1- dimensionali
    if sim.shape[0] - 1 > top_k - 1:  
      idx_partitioned = idx_partitioned[-(top_k - 1):]

    k['ids'] = self.ids[idx_partitioned].flatten()
    k['labels'] = self.labels[idx_partitioned].flatten()
    k['similarities'] = sim[idx_partitioned]
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, sim.size)
    return k
    

### Loading images DATA


In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 64

#images normalization 
sketches_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
mirflickr_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

sketches_generator = sketches_datagen.flow_from_directory(
        # This is the target directory
        SKETCHES_DIR + '/all',
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

mirflickr_generator = mirflickr_datagen.flow_from_directory(
        # This is the target directory
        MIRFLICKR_DIR,
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)


# unique, counts = np.unique(sketches_generator.classes, return_counts=True)
# labels_dict = dict(zip(unique, counts))
# print(labels_dict)

Found 20000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.


In [ ]:
def extract_features(extractor, generator, sample_count, dim=2048):
  features = np.zeros((sample_count, dim)) #extractor output shape 
  i = 0
  for inputs_batch, labels_batch in generator:
    start = time.time()
    features_batch = extractor.predict(inputs_batch)
    start = time.time()
    if (i + 1) * BATCH_SIZE > sample_count:
      features[i * BATCH_SIZE : sample_count , :] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : sample_count]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    else:
      features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE, : ] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    i += 1
    if i * BATCH_SIZE >= sample_count:
      break
  
  return features


## Evaluation

### Non Finetuned InceptionV3

In [ ]:
conv_base = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=INPUT_SHAPE,
                  pooling='avg')

87916544/87910968 [==============================] - 0s 0us/step


In [ ]:
sketches_features = extract_features(conv_base, sketches_generator, 20000)
mirflickr_features = extract_features(conv_base, mirflickr_generator, 25000)

In [ ]:
no_index_base = NO_INDEX()
no_index_base.insert(np.vstack((sketches_features, mirflickr_features)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features.shape[0]))))

In [ ]:
print(mAP(no_index_base, sketches_features))
print(mAP(no_index_base, sketches_features, mode = 'similarity'))

0.07724669044255322
0.0863004476614567


In [ ]:
del no_index_base
del sketches_features
del mirflickr_features

## Finetuned models

In [ ]:
MODEL_PATH = "/content/gdrive/Shareddrives/COMPUTER_VISION/models"

In [ ]:
def evaluate(model, feature_dim=2048, accuracy=True):
  if accuracy:
    acc_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = acc_datagen.flow_from_directory(
      SKETCHES_DIR + '/train',
      shuffle=True,
      target_size=(IMG_HEIGHT, IMG_WIDTH),
      batch_size=BATCH_SIZE)
    
    print('Train Accuracy')
    print(model.evaluate(train_generator, return_dict=True))

    test_generator = acc_datagen.flow_from_directory(
      SKETCHES_DIR + '/test',
      shuffle=True,
      target_size=(IMG_HEIGHT, IMG_WIDTH),
      batch_size=BATCH_SIZE)
    
    print('Test Accuracy')
    print(model.evaluate(test_generator, return_dict=True))

    extractor = Model(model.input, model.layers[-2].output)
  else:
    extractor = model
  
  sketches_generator = sketches_datagen.flow_from_directory(
        # This is the target directory
        SKETCHES_DIR + '/train',
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

  mirflickr_generator = mirflickr_datagen.flow_from_directory(
          # This is the target directory
          MIRFLICKR_DIR,
          shuffle=False,
          target_size=(IMG_HEIGHT, IMG_WIDTH),
          batch_size=BATCH_SIZE)

  sketches_features = extract_features(extractor, sketches_generator, 15000, dim=feature_dim)
  mirflickr_features = extract_features(extractor, mirflickr_generator, 25000, dim=feature_dim)

  no_index_base = NO_INDEX()
  no_index_base.insert(np.vstack((sketches_features, mirflickr_features)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features.shape[0]))))

  print('Euclidean mAP on train ')
  train_images_per_class = 80 - test_images_per_class
  print(mAP(no_index_base, sketches_features, img_per_labels=train_images_per_class))
  print('Cosine mAP on train')
  print(mAP(no_index_base, sketches_features, mode = 'similarity', img_per_labels=train_images_per_class))

  del sketches_features

  sketches_generator = sketches_datagen.flow_from_directory(
        # This is the target directory
        SKETCHES_DIR + '/test',
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

  sketches_features_test = extract_features(extractor, sketches_generator, 5000, dim=feature_dim)

  print('Euclidean mAP on train using test')
  print(mAP(no_index_base, sketches_features_test, img_per_labels=test_images_per_class, check_first_label=False, n_ground_truth=train_images_per_class))
  print('Cosine mAP on train using test')
  print(mAP(no_index_base, sketches_features_test, mode = 'similarity', img_per_labels=test_images_per_class, check_first_label=False, n_ground_truth=train_images_per_class))


### Model 1 - One Dense layer (1024) (Prima Training della parte "aggiunta", poi training sbloccando ultimi 2 blocchi di inception)

In [ ]:
model = models.load_model(MODEL_PATH + '/inception_finetuning_classification.h5')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, None, None, 3 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, None, None, 3 0           batch_normalization_188[0][0]    
______________________________________________________________________________________________

In [ ]:
evaluate(model, 1024)

Found 15000 images belonging to 250 classes.
Train Accuracy
235/235 [==============================] - 87s 365ms/step - loss: 1.2724 - acc: 0.6617
{'loss': 1.2724181413650513, 'acc': 0.6617333292961121}
Found 5000 images belonging to 250 classes.
Test Accuracy
79/79 [==============================] - 27s 343ms/step - loss: 1.2328 - acc: 0.6648
{'loss': 1.2328084707260132, 'acc': 0.6647999882698059}
Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Euclidean mAP on train 
0.1452075874862116
Cosine mAP on train
0.16870332551687733
Found 5000 images belonging to 250 classes.
Euclidean mAP on train using test
0.07632720919431449
Cosine mAP on train using test
0.0821815475170971


### Model 2 - 2 Dense Layer (2048) (Prima Training della parte "aggiunta", poi training sbloccando ultimi 2 blocchi di inception)

In [ ]:
model = models.load_model(MODEL_PATH + '/inception_finetuning_classification_2.h5')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, None, None, 3 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, None, None, 3 0           batch_normalization_470[0][0]    
____________________________________________________________________________________________

In [ ]:
evaluate(model)

Found 15000 images belonging to 250 classes.
Train Accuracy
235/235 [==============================] - 89s 337ms/step - loss: 1.1013 - acc: 0.7005
{'loss': 1.1013225317001343, 'acc': 0.7005333304405212}
Found 5000 images belonging to 250 classes.
Test Accuracy
79/79 [==============================] - 27s 343ms/step - loss: 1.1035 - acc: 0.6986
{'loss': 1.103475570678711, 'acc': 0.6985999941825867}
Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Euclidean mAP on train 
0.23363156631211146
Cosine mAP on train
0.31509726100797764
Found 5000 images belonging to 250 classes.
Euclidean mAP on train using test
0.2701175077785028
Cosine mAP on train using test
0.30359384222671254


### Model 3 - More train session on above model

In [ ]:
model = models.load_model(MODEL_PATH + '/inception_finetuning_classification_2_moretrain_3.h5')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, None, None, 3 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, None, None, 3 0           batch_normalization_470[0][0]    
____________________________________________________________________________________________

In [ ]:
evaluate(model)

Found 15000 images belonging to 250 classes.
Train Accuracy
235/235 [==============================] - 89s 337ms/step - loss: 0.9033 - acc: 0.7612
{'loss': 0.9033101797103882, 'acc': 0.7612000107765198}
Found 5000 images belonging to 250 classes.
Test Accuracy
79/79 [==============================] - 27s 340ms/step - loss: 0.9272 - acc: 0.7620
{'loss': 0.9271814227104187, 'acc': 0.7620000243186951}
Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Euclidean mAP on train 
0.3035769427659097
Cosine mAP on train
0.34601466850165147
Found 5000 images belonging to 250 classes.
Euclidean mAP on train using test
0.2889267980875142
Cosine mAP on train using test
0.34644290077427264


### Model 4 - Siamese (Using above "pretrained")

In [ ]:
model = models.load_model(MODEL_PATH + '/inception_finetuning_siamese_2.h5')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
model_3 (Functional)            (None, 2048)         34391840    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 2048)         0           model_3[0][0]              

In [ ]:
model = Model(model.layers[2].input, model.layers[-3].output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, None, None, 3 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, None, None, 3 0           batch_normalization_470[0][0]    
____________________________________________________________________________________________

In [ ]:
evaluate(model, accuracy=False) # accuracy on siamese notebook

Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Euclidean mAP on train 
0.26745477620900865
Cosine mAP on train
0.3175489602946406
Found 5000 images belonging to 250 classes.
Euclidean mAP on train using test
0.26186023701667976
Cosine mAP on train using test
0.32642290826888326


### Model 5 - 3 Dense Layer (2048) (Prima Training della parte "aggiunta", poi training sbloccando ultimi 2 blocchi di inception)

In [ ]:
model = models.load_model(MODEL_PATH + '/inception_finetuning_classification_3_last_one_more_train.h5')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, None, None, 3 96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, None, None, 3 0           batch_normalization_376[0][0]    
____________________________________________________________________________________________

In [ ]:
evaluate(model)

Found 15000 images belonging to 250 classes.
Train Accuracy
235/235 [==============================] - 85s 357ms/step - loss: 0.8940 - acc: 0.7667
{'loss': 0.8940337300300598, 'acc': 0.7667333483695984}
Found 5000 images belonging to 250 classes.
Test Accuracy
79/79 [==============================] - 27s 336ms/step - loss: 0.9065 - acc: 0.7610
{'loss': 0.9064938426017761, 'acc': 0.7609999775886536}
Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Euclidean mAP on train 
0.28967156650473397
Cosine mAP on train
0.35390787928538614
Found 5000 images belonging to 250 classes.
Euclidean mAP on train using test
0.29931062393476354
Cosine mAP on train using test
0.3784575093516286


### Model 6 - Model above with all parameters learnable (using above pretrained)

In [ ]:
model = models.load_model(MODEL_PATH + '/inception_finetuning_classification_3_last_one_more_train_all_parameters_more_train.h5')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, None, None, 3 96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, None, None, 3 0           batch_normalization_376[0][0]    
____________________________________________________________________________________________

In [ ]:
evaluate(model)

Found 15000 images belonging to 250 classes.
Train Accuracy
235/235 [==============================] - 85s 357ms/step - loss: 0.6899 - acc: 0.8087
{'loss': 0.6899421215057373, 'acc': 0.808733344078064}
Found 5000 images belonging to 250 classes.
Test Accuracy
79/79 [==============================] - 26s 332ms/step - loss: 1.0764 - acc: 0.7090
{'loss': 1.0763945579528809, 'acc': 0.7089999914169312}
Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Euclidean mAP on train 
0.4294365633329596
Cosine mAP on train
0.47119328533580646
Found 5000 images belonging to 250 classes.
Euclidean mAP on train using test
0.37318497794196365
Cosine mAP on train using test
0.4047550711955787


### Model 7 - 2 Dense Layer (4096)

In [ ]:
model = models.load_model(MODEL_PATH + '/inception_finetuning_classification_dense_only_4096.h5')
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, None, None, 3 96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, None, None, 3 0           batch_normalization_376[0][0]    
___________________________________________________________________________________________

In [ ]:
evaluate(model, 4096)

Found 15000 images belonging to 250 classes.
Train Accuracy
235/235 [==============================] - 88s 334ms/step - loss: 0.7600 - acc: 0.7867
{'loss': 0.7599719762802124, 'acc': 0.7866666913032532}
Found 5000 images belonging to 250 classes.
Test Accuracy
79/79 [==============================] - 27s 334ms/step - loss: 0.8022 - acc: 0.7840
{'loss': 0.8021575212478638, 'acc': 0.7839999794960022}
Found 15000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.
Euclidean mAP on train 
0.30681213848066224
Cosine mAP on train
0.4071802962057273
Found 5000 images belonging to 250 classes.
Euclidean mAP on train using test
0.293511345137655
Cosine mAP on train using test
0.3797378518794341
